# QuickPBSA Integration for AG Heine
Created by Andreas Brilka Februar 2025

### 0 Tutorial

* To get started, first select the appropriate kernel: agheine

![Selecting a kernel](tutorial_select_kernel.png)

![AG Heine kernel](tutorial_kernel_agheine.png)

* Now you have to run each cell by clicking the run icon on the top left side

![Running a cell](tutorial_run_cell.png)

* To copy the plots to the clipboard (left button) or save them (right button), click on the upper right side

![Save plot](tutorial_save_image.png)

* When you are done, don't forget to close the command shell window which also opened when you started this notebook. This will clean up temporary files

BUT: Do this only AFTER you closed the notebook

![Close the notebook](tutorial_close_notebook.png)

![Closing the cmd](tutorial_cmd.png)

### 1 Imports
You need to run this cell first to import all necessary dependencies

In [ ]:
# Imports
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import quickpbsa as pbsa
import tkinter as tk
from tkinter import filedialog
from pathlib import Path
import math

from IPython.core.display import HTML

root = tk.Tk()
root.withdraw()
root.call('wm', 'attributes', '.', '-topmost', True)

notebook_path = Path(os.path.abspath(""))

data, data_labels, results = None, None, None

### 2 Import data from ImageJ/Neurotorch Multimeasure or use example data

After running the first cell, you can either use example data (run section 2a) or import files from ImageJ/Neurotorch (run section 2b)

In [ ]:
# Defining plot function
data, data_labels = None, None
def plot_data(combine_plots = False, title:str|None = None):
    global data, data_labels
    if data is None or data_labels is None: return
    
    if combine_plots:
        plt.xlabel("Frame")
        plt.ylabel("Intensity")
        plt.title(title if title is not None else "")
        for i in range(data.shape[0]):
            plt.plot(range(data.shape[1]), data[i], label=data_labels[i])
        plt.legend()
        plt.show()
        return
            
    nrows, ncols = math.ceil(data.shape[0]/3), 3
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(15, 5*nrows))
    axes = axes.flatten()
    for ax in axes:
        ax.set_axis_off()
    for i in range(data.shape[0]):
        ax = axes[i]
        ax.set_axis_on()
        x, y = range(data.shape[1]), data[i]
        ax.plot(x,y)
        ax.set_xlabel("Frame")
        ax.set_ylabel("Intensity")
        ax.set_title(data_labels[i])
    fig.tight_layout()
    plt.show()

#### 2a Load example data
Run the following cell to load example data

In [ ]:
# Load and plot xample data
data = np.loadtxt(notebook_path / 'ExampleTrace.txt')
data = np.append(data, [np.sum(data, axis=0)], axis=0)
data_labels = ['4 Fluorophores','3 Fluorophores','3 Fluorophores','Sum']

plot_data(combine_plots=True, title="Example Dataset")

#### 2b Load multimeasure.csv from ImageJ/Neurotorch

In [ ]:
# Load data from a multi measure file
if not (p := filedialog.askopenfilename(parent=root, title="Open a multi measure CSV file", defaultextension="*.csv", filetypes=[(".csv", "*.csv"), ("All files", "*.*")])) or not (path := Path(p)).exists():
    print("Your provided path is invalid or empty")
else:
    tempFile = notebook_path / "file.csv"
    data, data_labels = None, None
    data_multimeasure = pd.read_csv(path, sep=",", header=0, index_col=0)
    display(HTML(f"<h3> {path.name} </h3>"))
    display(data_multimeasure)

    data = data_multimeasure.to_numpy().transpose()
    data_labels = list(data_multimeasure.columns)

    plot_data()

### 3 Detect steps by using quickPBSA

You need to set the settings in the following cell by hand. Then you run the two cells below to plot the result

In [ ]:
# Settings: This values needs to be changed!
KV_threshold = 20 # KV threshold (1/3 to 1/2 of typical last step)
KV_maxiter = 100   # Maximum KV iterations (much higher than expected number of steps)

Now run this cell once to define the plot function

In [ ]:
# Defining a plot function for the result
def plot_results():
    global data, data_labels, results
    if data is None or data_labels is None or results is None or len(results) == 0: return

    nrows, ncols = math.ceil(data.shape[0]), 4
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(23, 5*nrows))
    axes = axes.flatten()
    for i in range(data.shape[0]):
        r = results[i]
        ax1 = axes[4*i]
        ax2 = axes[4*i+1]
        ax3 = axes[4*i+2]
        ax4 = axes[4*i+3]
        ax2_twin = ax2.twinx()
        ax3_twin = ax3.twinx()
        ax4_twin = ax4.twinx()
        for ax in [ax1, ax2, ax3, ax4]:
            ax.plot(data[i], label="Trace")
        for ax in [ax1, ax4]:
            ax.plot(r["meantrace"], c="orange", label="Means")
        for ax_twin in [ax2_twin, ax4_twin]:
            ax_twin.plot(r["fluortrace_primary"], "m--", label="Fluorophores (primary refinement)")
        for ax_twin in [ax3_twin, ax4_twin]:
            if not r["fluortrace_secondary"] is None:
                ax_twin.plot(r["fluortrace_secondary"], "r--", label="Fluorophores (secondary refinement)")
        for ax, ax_twin in zip([ax1, ax2, ax3, ax4], [None, ax2_twin, ax3_twin, ax4_twin]):
            ax.set_xlabel("Frame")
            ax.set_ylabel("Intensity")
            ax.set_title(data_labels[i])
            if ax_twin is not None:
                ax_twin.yaxis.set_major_locator(MaxNLocator(integer=True))
                ax_twin.set_ylabel('Fluorophores', c="red")
                lines, labels = ax.get_legend_handles_labels()
                lines2, labels2 = ax_twin.get_legend_handles_labels()
                ax_twin.legend(lines + lines2, labels + labels2, loc=0)
            else:
                ax.legend()
    fig.tight_layout()
    plt.show()

Rerun the following cell every time you want to replot data

In [ ]:
# Detect steps using quickPBSA

results = []

for i in range(data.shape[0]):
    trace = data[i]
    trace_flipped = np.flipud(trace)
    steppos, means, variances, posbysic, niter = pbsa.steps_preliminary.kv_single_fast(trace_flipped, KV_threshold, KV_maxiter)
    
    diffs = np.diff(np.hstack([0, steppos, len(trace_flipped)]))
    meantrace = np.repeat(means, diffs)
    fluors = np.cumsum(np.hstack((0, np.sign(np.diff(means)))))
    fluortrace = np.repeat(fluors, diffs)

    try:
        success, sicmin, steppos_out, step_out = pbsa.steps_refinement.improve_steps_single(trace_flipped, steppos, means, variances, posbysic)
    except Exception as ex:
        print(f"Failed secondary refinment step due to the following error:", str(ex))
        success, sicmin, steppos_out, step_out = None, None, None, None
        diffs2, fluors2, fluortrace2 = None, None, None
    else:
        diffs2 = np.diff(np.hstack([0, steppos_out, len(trace)]))
        fluors2 = np.cumsum(np.hstack([0, step_out]))
        fluortrace2 = np.repeat(fluors2, diffs2)

        steppos_out = np.flipud(steppos_out)
        step_out = np.flipud(step_out)
        diffs2 = np.flipud(diffs2)
        fluors2 = np.flipud(fluors2)
        fluortrace2 = np.flipud(fluortrace2)

    steppos = np.flipud(steppos)
    means = np.flipud(means)
    variances = np.flipud(variances)
    posbysic = np.flipud(posbysic)
    diffs = np.flipud(diffs)
    meantrace = np.flipud(meantrace)
    fluors = np.flipud(fluors)
    fluortrace = np.flipud(fluortrace)

    results.append({"steppos": steppos, "means": means, "variances": variances, "posbysic": posbysic, "niter": niter, "success": success, "sicmin": sicmin, "steppos_out": steppos_out, "step_out": step_out, 
                    "diffs_primary": diffs, "meantrace": meantrace, "fluors_primary": fluors, "fluortrace_primary": fluortrace, "diffs_secondary": diffs2, "fluors_secondary": fluors2, "fluortrace_secondary": fluortrace2})
    
plot_results()